In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import my_function as my

### 실적기준월

In [2]:
yyyymm = '202006'

### Data - fanace : pkl활용

### 현 보유 포트(+순위) 정보 

In [3]:
# finance_file = 'data/재무_'+yyyymm+'.xlsx'
# my.get_finance(finance_file).to_pickle('pkl/finance.pkl')

In [4]:
port_except = ['TIGER 소프트웨어','KODEX 바이오','현대차','태림포장']
port = my.get_port( port_except, yyyymm, 500)
port = port.sort_values('투자기준월').reset_index(drop=True)
print(port.shape)
port # 현재가 표기 안하기로

(24, 12)


,투자기준월,종목코드,종목명,매수가격,체결잔고,매수금액,평가금액,평가손익,수익률,보유여부,최종순위,현재가
0,,000990,DB하이텍,20937,24,502500,896400,393900,78.39,보유중,3.0,37350.0
1,,002460,화성산업,12450,40,498000,484000,-14000,-2.81,보유중,7.0,12100.0
2,,004000,롯데정밀화학,48550,10,485500,529000,43500,8.96,보유중,16.0,52900.0
3,,007540,샘표,52200,12,626400,585600,-40800,-6.51,보유중,22.0,48800.0
4,,007700,F&F,97860,5,489300,477000,-12300,-2.51,보유중,7.0,95400.0
5,,017960,한국카본,8370,60,502200,615000,112800,22.46,보유중,13.0,10250.0
6,,035150,백산,5600,89,498400,400945,-97455,-19.55,보유중,7.0,4505.0
7,,051600,한전KPS,32203,15,483050,408000,-75050,-15.54,보유중,6.0,27200.0
8,,053210,스카이라이프,9790,66,646140,652740,6600,1.02,보유중,24.0,9890.0
9,,069260,휴켐스,19353,26,503200,620100,116900,23.23,보유중,30.0,23850.0


### [매도-기간기준] 투자기준월 가장 오래된 2종목

In [5]:
closing = port.sort_values('투자기준월') [['투자기준월','종목코드','종목명','최종순위','평가손익','수익률','현재가','체결잔고','매수금액']]
closing.head(2)

,투자기준월,종목코드,종목명,최종순위,평가손익,수익률,현재가,체결잔고,매수금액
0,,000990,DB하이텍,3.0,393900,78.39,37350.0,24,502500
21,,001880,대림건설,1.0,-6000,-1.19,24850.0,20,503000


### [매도-순위기준] : 24위 순위 밖

In [6]:
sell = port[port['최종순위'] > 24] [['투자기준월','종목코드','종목명','최종순위','평가손익','수익률','현재가','체결잔고','평가금액']]
sell.sort_values('최종순위')

,투자기준월,종목코드,종목명,최종순위,평가손익,수익률,현재가,체결잔고,평가금액
23,,294870,HDC현대산업개발,26.0,-29700,-6.07,20900.0,22,459800
9,,069260,휴켐스,30.0,116900,23.23,23850.0,26,620100


In [7]:
base_amt = 650000

### [추가매수] 증분만 더 사기

In [8]:
stock_nm = '조선선재'
print("추가매수 : ", stock_nm)
round((base_amt - port[port['종목명'] == stock_nm]['매수금액']) / port[port['종목명'] == stock_nm]['현재가'],1)

추가매수 :  조선선재


10    1.8
dtype: float64

### [매수] : 미보유 종목 중 topn(24종목 기준, 매도대상 고려)

In [9]:
import math
sell_cnt = 24 - (len(port) - sell.shape[0])
sell_cnt = 5
buy = my.get_stock_topn(sell_cnt, yyyymm, 500, 1)
buy['매수가능'] = round(base_amt / buy['현재가'],1)
buy['매수금액'] = buy['현재가'] * buy['매수가능'].apply(lambda x : math.floor(x))
buy[['종목코드','종목명','최종순위','현재가','매수가능','매수금액']].reset_index(drop=True)

,종목코드,종목명,최종순위,현재가,매수가능,매수금액
0,003230,삼양식품,16.0,102000.0,6.4,612000.0
1,002960,한국쉘석유,23.0,239500.0,2.7,479000.0
2,002310,아세아제지,25.0,28750.0,22.6,632500.0
3,035000,지투알,26.0,5240.0,124.0,649760.0
4,005960,동부건설,26.0,11200.0,58.0,649600.0


### (참고) 전체 종목 기준으로 top24 살펴보기

In [10]:
my.get_stock_topn(24, yyyymm, 500, 0)

,종목코드,종목명,자산총계,부채총계,최근4Q,시가총액,현재가,자본수익률,이익수익률,자본수익률순위,이익수익률순위,순위합산,최종순위,매수금액
0,001880,대림건설,9576.6,4157.5,1775.7,5480.0,24850.0,0.19,0.18,7.0,3.0,10.0,1.0,503000
1,002170,삼양통상,3750.1,370.2,544.2,1941.0,64700.0,0.15,0.24,13.0,1.0,14.0,2.0,494500
2,000990,DB하이텍,11231.1,3831.7,2512.9,16583.0,37350.0,0.22,0.12,2.0,20.0,22.0,3.0,502500
3,248170,샘표식품,2356.7,615.7,368.4,2079.0,45500.0,0.16,0.14,12.0,11.0,23.0,4.0,513300
4,229640,LS전선아시아,3140.5,123.9,376.3,2358.0,7700.0,0.12,0.15,29.0,4.0,33.0,5.0,502500
5,051600,한전KPS,13134.0,2625.6,1821.1,12240.0,27200.0,0.14,0.12,19.0,20.0,39.0,6.0,483050
6,007700,F&F,6096.0,1030.4,1524.1,14692.0,95400.0,0.25,0.10,1.0,39.0,40.0,7.0,489300
7,035150,백산,2749.4,1187.8,330.1,1090.0,4505.0,0.12,0.14,29.0,11.0,40.0,7.0,498400
8,002460,화성산업,5303.3,1389.2,569.6,1507.0,12100.0,0.11,0.20,38.0,2.0,40.0,7.0,498000
9,134380,미원화학,1348.3,260.3,180.7,1232.0,56000.0,0.13,0.12,23.0,20.0,43.0,10.0,505100
